In [375]:
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [380]:
dfjointe = pd.read_csv('Bases de données/donnéesjointes.csv')
dfjointe.drop('Unnamed: 0', axis=1, inplace=True)

path = 'Bases de données/revenus.csv'
dfrev = pd.read_csv(path, sep = ';')
rename = {'CodePostal' : 'Code postal', ' Niveau de vie Commune ':'Niveau de vie commune', ' NiveaudevieDépartement ':'Niveau de vie département', 'Departement' : 'Département'}
dfrev = dfrev.rename(rename, axis= 1)

#On garde les colonnes qui nous intéresse
dfrev.drop(['Code postal', 'Niveau de vie département'], axis=1, inplace=True)
dfrev.set_index('Nom Commune', inplace=True)
dfjointe = dfjointe.join(dfrev, on='Localisation', how='left')
dfjointe.reset_index(inplace=True)
dfjointe.drop_duplicates(subset='index', inplace=True)
dfjointe.set_index('index', inplace=True)

In [381]:
dfjointe

,Prix (€),Marque,Taille,Etat,Matière,Localisation,Option de paiement,Vues,Date d'ajout,Département,Libellé du département,Nbvoix,%popent,%popvot,Niveau de vie commune
index,,,,,,,,,,,,,,,
0,6.0,NaN,W34 | FR 44,Très bon état,NaN,Tourny,Carte bancaire,13,"12/28/2023, 6:19:17 PM",27.0,Eure,53761.0,12.39,25.92,21 250
1,5.0,Zara,W32 | FR 42,Satisfaisant,NaN,Riantec,Carte bancaire,2,"12/29/2023, 10:43:33 AM",56.0,Morbihan,82148.0,13.44,26.09,20 710
2,6.0,Zara,W30 | FR 40,Très bon état,NaN,Feignies,Carte bancaire,1,"12/29/2023, 11:39:14 AM",59.0,Nord,232470.0,12.77,29.29,16 860
3,5.0,Leeyo Jeans,W28 | FR 38,Très bon état,NaN,Buigny-Saint-Maclou,Carte bancaire,0,"12/29/2023, 12:51:07 PM",80.0,Somme,52514.0,12.78,26.61,21 846
4,5.0,Casual by Gémo,W38 | FR 48,Neuf sans étiquette,NaN,Fougueyrolles,Carte bancaire,1,"12/29/2023, 11:09:58 AM",24.0,Dordogne,43986.0,13.77,25.77,18 238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,5.0,H&M,W23 | FR 32,Très bon état,NaN,Paris,Carte bancaire,12,"12/27/2023, 12:27:55 PM",75.0,Paris,298944.0,21.94,40.16,NaN
1392,9.0,AARHON,W30 | FR 40,Neuf sans étiquette,NaN,Esternay,Carte bancaire,5,"12/29/2023, 1:50:51 PM",51.0,Marne,37475.0,9.80,21.94,18 285
1393,5.0,SMOG,W36 | FR 46,Bon état,NaN,Le Mans,Carte bancaire,6,"12/28/2023, 8:43:05 PM",72.0,Sarthe,46201.0,11.21,25.23,19 113


In [382]:
#Pas de prise en compte des marques car non exhaustif, de la matière car trop peu de données,
#d'option de paiement car tjrs carte bleue, pas le département car déja décrit par le revenu
#et les votes
dfmodel = pd.DataFrame()
dfmodel = dfjointe[['Prix (€)', 'Taille', 'Etat', 'Vues', '%popvot', 'Niveau de vie commune']]
dfmodel = dfmodel.dropna()

dfmodel['Niveau de vie commune'] = dfmodel['Niveau de vie commune'].apply(lambda x: x.replace(' ',''))
dfmodel['Niveau de vie commune'] = pd.to_numeric(dfmodel['Niveau de vie commune'])
dfmodel['Vues'] = pd.to_numeric(dfmodel['Vues'])

#transformation des variables catégorielles, jean de base est 5XL et bon état
dummiesEtat = pd.get_dummies(dfmodel['Etat'], drop_first=True, dtype=int)
dummiesTaille = pd.get_dummies(dfmodel['Taille'], drop_first=True, dtype=int)
dfmodel.drop(['Taille', 'Etat'], axis=1, inplace=True)
dfmodeldummies = pd.concat([dfmodel, dummiesTaille, dummiesEtat], axis=1)

In [383]:
dfmodeldummies['Prix (€)'].value_counts()

Prix (€)
5.00     861
6.00     155
8.00      78
10.00     61
7.00      38
9.00      31
12.00     25
5.95       5
15.00      3
7.50       2
5.50       2
8.99       2
Name: count, dtype: int64

In [384]:
dfmodeldummies.columns

Index(['Prix (€)', 'Vues', '%popvot', 'Niveau de vie commune', 'L', 'M', 'S',
       'W23 | FR 32', 'W24 | FR 34', 'W25 | FR 34', 'W26 | FR 36',
       'W27 | FR 36', 'W28 | FR 38', 'W29 | FR 38', 'W30 | FR 40',
       'W31 | FR 40', 'W32 | FR 42', 'W33 | FR 42', 'W34 | FR 44',
       'W36 | FR 46', 'W38 | FR 48', 'W40 | FR 50', 'W48 | FR 58', 'XL',
       'XXS / 32 / 4', 'Neuf avec étiquette', 'Neuf sans étiquette',
       'Satisfaisant', 'Très bon état'],
      dtype='object')

In [428]:
#Pas de stratification par les communes possibles car trop peu de sorties de prix
X_train, X_test, Y_train, Y_test = train_test_split(dfmodeldummies[['Vues', '%popvot', 'Niveau de vie commune',
                                                                    'L', 'M', 'S',
                                                                    'W23 | FR 32',
                                                                    'W24 | FR 34',
                                                                    'W25 | FR 34',
                                                                    'W26 | FR 36',
                                                                    'W27 | FR 36',
                                                                    'W28 | FR 38',
                                                                    'W29 | FR 38',
                                                                    'W30 | FR 40',
                                                                    'W31 | FR 40',
                                                                    'W32 | FR 42',
                                                                    'W33 | FR 42',
                                                                    'W34 | FR 44',
                                                                    'W36 | FR 46',
                                                                    'W38 | FR 48',
                                                                    'W40 | FR 50',
                                                                    'W48 | FR 58',
                                                                    'XL',
                                                                    'XXS / 32 / 4',
                                                                    'Neuf avec étiquette',
                                                                    'Neuf sans étiquette',
                                                                    'Satisfaisant',
                                                                    'Très bon état'
                                                                   ]],
                                                    dfmodeldummies['Prix (€)'],
                                                    train_size=0.75,
                                                   )

In [429]:
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

normalizer = preprocessing.Normalizer()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

In [430]:
reg = LinearRegression()
reg.fit(X_train, Y_train)

LinearRegression()

In [431]:
#calcul du R²
reg.score(X_test, Y_test)

0.2511545390872745

In [432]:
coeffs = pd.DataFrame(index=dfmodeldummies.drop('Prix (€)', axis=1).columns)
coeffs['Coefficients']=reg.coef_
coeffs.sort_values('Coefficients').transpose()

,Niveau de vie commune,Satisfaisant,%popvot,Très bon état,W48 | FR 58,XXS / 32 / 4,Neuf sans étiquette,W25 | FR 34,W24 | FR 34,XL,...,W28 | FR 38,W26 | FR 36,W38 | FR 48,W34 | FR 44,W36 | FR 46,W30 | FR 40,W31 | FR 40,W33 | FR 42,L,W32 | FR 42
Coefficients,-1.578352,-1.005752,-0.776564,-0.508229,0.170926,0.273598,0.412383,0.531834,0.539889,0.820885,...,2.474678,2.753891,2.864789,2.936451,3.021122,3.475412,3.511885,4.34322,5.01672,5.119694


In [433]:
#Pas de stratification par les communes possibles car trop peu de sorties de prix
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(dfmodeldummies[['Vues', '%popvot', 'Niveau de vie commune',
                                                                    'Neuf avec étiquette',
                                                                    'Neuf sans étiquette',
                                                                    'Satisfaisant',
                                                                    'Très bon état'
                                                                   ]],
                                                    dfmodeldummies['Prix (€)'],
                                                    train_size=0.75,
                                                   )

In [434]:
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)

normalizer = preprocessing.Normalizer()
X_train2 = normalizer.fit_transform(X_train2)
X_test2 = normalizer.transform(X_test2)

In [435]:
reg2 = LinearRegression()
reg2.fit(X_train2, Y_train2)

LinearRegression()

In [436]:
reg2.score(X_test2, Y_test2)

0.12452400779889738

In [437]:
coeffs2 = pd.DataFrame(index=dfmodeldummies.drop(['Prix (€)', 'L', 'M', 'S',
                                                                    'W23 | FR 32',
                                                                    'W24 | FR 34',
                                                                    'W25 | FR 34',
                                                                    'W26 | FR 36',
                                                                    'W27 | FR 36',
                                                                    'W28 | FR 38',
                                                                    'W29 | FR 38',
                                                                    'W30 | FR 40',
                                                                    'W31 | FR 40',
                                                                    'W32 | FR 42',
                                                                    'W33 | FR 42',
                                                                    'W34 | FR 44',
                                                                    'W36 | FR 46',
                                                                    'W38 | FR 48',
                                                                    'W40 | FR 50',
                                                                    'W48 | FR 58',
                                                                    'XL',
                                                                    'XXS / 32 / 4'], axis=1).columns)

In [438]:
coeffs2['Coefficients']=reg2.coef_
coeffs2.sort_values('Coefficients').transpose()

,Satisfaisant,Niveau de vie commune,%popvot,Très bon état,Neuf sans étiquette,Vues,Neuf avec étiquette
Coefficients,-1.019933,-0.938826,-0.387263,-0.105942,0.068459,0.827768,1.04108
